Perusahaan peminjaman sepeda di San Fransisco memintamu untuk menganalisa durasi perjalanan customer mereka. Untuk memudahkan analisa, lakukan hal-hal berikut ini:

STATISTIK DESKRIPTIF
Lakukan perhitungan central tendency (mean, median, dan modus).
Lakukan pengecekan dispersi (standar deviasi, variance, dan range).
Lakukan pengecekan skewness dan kurtosis.
Analisa apakah ada data outlier berdasarkan sebaran datanya.
Cari tahu berapa persentase outlier terhadap data keseluruhan.
Bagaimana perubahan pemusatan data sebelum dan sesudah penghapusan outlier?
API
Buatlah API untuk:

Menampilkan seluruh entry data setelah dilakukan handling outlier.
Menghapus entry data, dari data yang telah dihandling outlier.
clue:

Load data csv yang sudah diolah dengan pandas, kemudian konversi data ke dictionary df.to_dict() atau json df.to_json() untuk dapat diolah lebih lanjut dengan API menggunakan FastAPI.
Pembuatan API dilakukan pada script .py
Endpoint dan penamaan function pada API dibebaskan.
Data dapat diakses dari BigQuery dengan ketentuan:

Project_id: bigquery-public-data
Dataset: san_francisco_bikeshare
Table: bikeshare_trips
Ambil hanya kolom duration_sec saja
Berikan LIMIT pada data yang diambil sebesar 3000 entry data
Koneksikan data di atas menggunakan BigQuery ke Google Colab dengan code berikut:

from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

  project_id = "rock-wonder-317907" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)
Untuk melakukan Query menggunakan cara ini, kamu dapat menggunakan method client.query('Masukkan Querynya').to_dataframe(). Outputnya akan berupa Pandas dataframe, sehingga harus import Pandas.

Contoh:

df = client.query('''
SELECT *
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at < "2022-07-01"
ORDER BY year,month ASC
''').to_dataframe()

In [ ]:
# Working Area
import pandas as pd
import numpy as np
from scipy import stats

# Membuat dataset dummy
np.random.seed(42)
duration_sec = np.concatenate([
    np.random.normal(600, 200, 2700),  # durasi normal
    np.random.normal(1800, 300, 270),  # durasi panjang
    np.random.normal(200, 50, 30)      # durasi pendek
])
df = pd.DataFrame({'duration_sec': duration_sec})

# 1. Central Tendency
mean = df['duration_sec'].mean()
median = df['duration_sec'].median()
mode = df['duration_sec'].mode().values[0]

print("1. Central Tendency:")
print(f"Mean: {mean:.2f}")
print(f"Median: {median:.2f}")
print(f"Mode: {mode:.2f}")

# 2. Dispersi
std_dev = df['duration_sec'].std()
variance = df['duration_sec'].var()
range_val = df['duration_sec'].max() - df['duration_sec'].min()

print("\n2. Dispersi:")
print(f"Standard Deviation: {std_dev:.2f}")
print(f"Variance: {variance:.2f}")
print(f"Range: {range_val:.2f}")

# 3. Skewness dan Kurtosis
skewness = df['duration_sec'].skew()
kurtosis = df['duration_sec'].kurtosis()

print("\n3. Skewness dan Kurtosis:")
print(f"Skewness: {skewness:.2f}")
print(f"Kurtosis: {kurtosis:.2f}")

# 4. Deteksi Outlier menggunakan IQR
Q1 = df['duration_sec'].quantile(0.25)
Q3 = df['duration_sec'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['duration_sec'] < lower_bound) | (df['duration_sec'] > upper_bound)]

print("\n4. Outliers:")
print(f"Jumlah outlier: {len(outliers)}")

# 5. Persentase Outlier
percentage_outliers = (len(outliers) / len(df)) * 100

print("\n5. Persentase Outlier:")
print(f"Persentase outlier: {percentage_outliers:.2f}%")

# 6. Perubahan pemusatan data sebelum dan sesudah penghapusan outlier
df_clean = df[(df['duration_sec'] >= lower_bound) & (df['duration_sec'] <= upper_bound)]

print("\n6. Perubahan pemusatan data:")
print("Sebelum penghapusan outlier:")
print(f"Mean: {df['duration_sec'].mean():.2f}")
print(f"Median: {df['duration_sec'].median():.2f}")

print("\nSetelah penghapusan outlier:")
print(f"Mean: {df_clean['duration_sec'].mean():.2f}")
print(f"Median: {df_clean['duration_sec'].median():.2f}")

# Menyimpan data yang sudah dibersihkan
df_clean.to_csv('clean_bike_share_data.csv', index=False)